In [1]:
import pandas as pd
import requests, json
import numpy as np
import folium
import re
import json
import datetime
import xml.etree.ElementTree as Xml

In [2]:
minlon, minlat, maxlon, maxlat = 77.7519486464,12.9833797192,77.7528560462,12.9842016832

minlat=str(minlat)
minlon=str(minlon)
maxlat=str(maxlat)
maxlon=str(maxlon)

In [3]:
hermap_filename = 'data_comparision/heremap.json'
sumo_filename = 'data_comparision/original_sumo.net.xml'

In [4]:
with open(hermap_filename) as f:
    heremap_json = json.load(f)

In [37]:
get_transform(1789.01,2940.24)

(77.7521041995935, 12.983818048330702)

In [82]:
def get_transform(list_point):
    x = list_point[0]
    y = list_point[1]
    loc = [77.735568,12.957348,77.757670,12.994528]
    points = [0.00,-0.00,2391.16,4129.88]
    l1 = loc[0]+((loc[2]-loc[0])/(points[2]-points[0]))*x
    l2 = loc[1]+((loc[3]-loc[1])/(points[3]-points[1]))*y
    return [l2,l1]

In [116]:
e = Xml.parse(sumo_filename).getroot()
osm=folium.Map(location=[12.96984,77.74992],zoom_start=14)

for i in e:
    if i.tag == 'edge':
        #print(i.attrib['shape'])
        
        for j in i:
            list_points = []
            if j.attrib['id'] in ["122516806_0", "122516806_1", "143215059_0" ,"143215059_1" ,"580921886#0_0" ,"580921886#0_1" ,"28043853_0" ,"28043853_1"]:
                print(j.attrib['id'])
                for ll in j.attrib['shape'].split():
                    list_points.append(get_transform([float(i) for i in ll.split(',')]))
            folium.PolyLine(locations=list_points,opacity=0.75,weight=4).add_to(osm)

122516806_0
122516806_1
143215059_0
143215059_1
28043853_0
28043853_1
580921886#0_0
580921886#0_1


In [117]:
        
osm

In [84]:
list_points

[[12.983818048330702, 77.7521041995935],
 [12.983820118947767, 77.75208802397164],
 [12.983817598196557, 77.75207101646063],
 [12.983810576103906, 77.75205326949263],
 [12.983798872616154, 77.75203469063551],
 [12.983818048330702, 77.7521041995935],
 [12.98383884452817, 77.75203145551113],
 [12.983846586835453, 77.75211251848475],
 [12.983867383032921, 77.75204005169876],
 [12.98391590749368, 77.75213294598439],
 [12.98391590749368, 77.75213294598439],
 [12.983887639069417, 77.75212351790763],
 [12.983887639069417, 77.75212351790763],
 [12.983712266806783, 77.7526237605681],
 [12.983706685143394, 77.75259630822697],
 [12.983696512111733, 77.7525749564061],
 [12.983681927765456, 77.75255970510547],
 [12.983662842077736, 77.75255046189298],
 [12.983712266806783, 77.7526237605681],
 [12.983706505089737, 77.75256589805785],
 [12.983705874901935, 77.7525217155021],
 [12.983712356833614, 77.7524780875391],
 [12.98372766139452, 77.75242244339987],
 [12.983742515821284, 77.75262644109972],
 [1

In [ ]:
def create_data_1(dictionary):
        
    RWS=dictionary['RWS']
    l=[]
    for rw in RWS:
        EBU_COUNTRY_CODE=rw['EBU_COUNTRY_CODE']
        EXTENDED_COUNTRY_CODE=rw['EXTENDED_COUNTRY_CODE']
        UNITS=rw['UNITS']
        for elem in rw['RW']:
            DE=elem['DE']
            LI=elem['LI']
            PBT=elem['PBT']
            mid=elem['mid']
            FIS=elem['FIS']
            for FI in FIS:
                elem_1=FI['FI']
                for elem_2 in elem_1:
                    LE=elem_2['TMC']['LE']
                    PC=elem_2['TMC']['PC']
                    QD=elem_2['TMC']['QD']
                    SHP=elem_2['SHP']
                    CF=elem_2['CF']
                    for elem_3 in CF:
                        FF=elem_3['FF']
                        CN=elem_3['CN']                    
                        JF=elem_3['JF']
                        SP=elem_3['SP']
                        TY=elem_3['TY']
                        l.append([EBU_COUNTRY_CODE,EXTENDED_COUNTRY_CODE,UNITS,DE,LI,PBT,mid,LE,PC,QD,SHP,FF,CN,JF,SP,TY])
    clms=['EBU_COUNTRY_CODE','EXTENDED_COUNTRY_CODE','UNITS','DE','LI','PBT','mid','LE','PC','QD','SHP','FF','CN','JF','SP','TY']                    
    df=pd.DataFrame(l,columns=clms)
    df['PBT']=datetime.datetime.now()
    #print(json.dumps(rws_list, indent=2))
#     if path_to_save:
#         if os.path.isfile(path_to_save):
#             df.to_csv(path_to_save,mode='a+',header=False,index=False)
#         else:
#             df.to_csv(path_to_save,mode='w+',index=False)
#     else:
    df['SHP'] = df['SHP'].astype(str)
    return df  

In [6]:
def traffic_map_across_city_historical(traffic_data):
    
    traffic_hour= traffic_data
    df_jaipur=traffic_data
    df_ori=df_jaipur
    list_shp = df_jaipur['SHP'].unique()
    list_mean = []
    start_point=[]
    end_point=[]
    for s in list_shp:
        tmp = re.findall("[+-]?\d*\.\d+", s)
        #print(tmp)
        tmp=[float(i) for i in tmp]
        tmp2 = []
        it = iter(tmp)
        for z in it:
            tmp2.append([z, next(it)])
        a = np.array(tmp2)
        list_mean.append(list(np.mean(a, axis=0)))
        start_point.append(tmp2[0])
        end_point.append(tmp2[-1])
    df_t = pd.DataFrame()
    df_t['SHP'] = list_shp
    df_t['mean'] = list_mean
    df_final =df_jaipur.merge(df_t,on='SHP')
    df_final['latitude'] = np.array(df_final['mean'].values.tolist())[:,0]
    df_final['longitude'] = np.array(df_final['mean'].values.tolist())[:,1]
    hh=pd.pivot_table(df_final,index=['SHP','PC','DE'],values='JF',aggfunc='median').reset_index()
    da1=hh.sort_values('JF',ascending=False)
    #da2=da1[da1['JF']>5]
    da2=da1

    data_2=da2
    data_2=data_2
    import branca.colormap as cm

    colormap = cm.LinearColormap(['green','yellow','red'],vmin=0,vmax=10)
    colormap.caption="Jamfactor_levels"
#     lat_long=df[['latitude','longitude']].values[0].tolist()
    x=folium.Map(location=[12.9380131671,77.6319900345],zoom_start=10)
    # x

    for i in data_2.index.tolist():
        color_grad_val=data_2['JF'][i]
        tmp2=[]
        #tmp=re.findall("\d*\.\d+", data_2['SHP'][i])
        tmp = re.findall("[+-]?\d*\.\d+", data_2['SHP'][i])
        #print(tmp)
        tmp=[float(i) for i in tmp]

        it = iter(tmp)
        for z in it:
            tmp2.append([z, next(it)])
        y=folium.PolyLine(locations=tmp2,color=colormap.rgb_hex_str(color_grad_val),opacity=0.75,weight=7)
        x.add_children(y)
        
    
    df= df_final[['latitude','longitude','PC']].drop_duplicates()

    l = df[['latitude','longitude']].values.tolist()

    #map_osm = folium.Map(location=[38.946241233265095, -77.04115322627564],tiles='CartoDB Positron',
    #                     zoom_start=12)
    for i,k in zip(l,df['PC']):

        marker = folium.Marker(i,popup=str(k),icon=folium.Icon())

        x.add_child(marker)

    

    x
    display(x)

In [13]:
heremap_json

NameError: name 'df' is not defined

In [22]:
traffic_map_across_city(heremap_data,'JF')

In [114]:
def traffic_map_across_city(x,df,label):
        #hh=pd.pivot_table(df,index=['SHP','PC','DE'],values='JF',aggfunc='median').reset_index()[:1000]
        #hh = df
        data_2=df#[df.QD == '-']
        if label == 'JF':
            colormap = folium.LinearColormap(['green','yellow','red'],vmin=0,vmax=10)
        else:
            colormap = folium.LinearColormap(['yellow','red'],vmin=df[label].min(),vmax=df[label].max())
        colormap.caption=label+"_levels"

        #x=folium.Map(location=[12.96984,77.74992],zoom_start=14)
        # x
        #x.add_child(colormap)
        for i in data_2.index.tolist():
            color_grad_val=data_2[label][i]
            tmp2=[]
            tmp=re.findall("\d*\.\d+", data_2['SHP'][i])
            tmp=[float(i) for i in tmp]

            it = iter(tmp)
            for z in it:
                tmp2.append([z, next(it)])

            y=folium.PolyLine(locations=tmp2,color=colormap.rgb_hex_str(color_grad_val),opacity=0.75,weight=4)
            x.add_child(y)
        return x

In [118]:
#heremap_data = create_data_1(heremap_json)
traffic_map_across_city(osm,heremap_data,'JF')

In [9]:
a = np.array([1,2,30,-1])
a

array([ 1,  2, 30, -1])

In [12]:
np.clip(a,0,10,a)

array([ 1,  2, 10,  0])